## Spark streaming test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *

creds, config = read_files()
subreddit = config["subreddit"]
kafka_host = config["kafka_host"]
spark_host = config["spark_host"]
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]

print("imported modules")

imported modules


In [2]:
try:
    main()

except KeyboardInterrupt:
    print("stopped spark streaming.")

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fffc9b05-4f8d-485f-ac61-7e6bd4b88b55;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

created spark successfully


22/05/11 22:43:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/05/11 22:43:38 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/05/11 22:43:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


## Debug main function

In [19]:
spark, sc = init_spark()

created spark successfully


In [18]:
stage_df = read_kafka_stream(spark, sc)

In [4]:
try:
    write_stream(stage_df)

except KeyboardInterrupt:
    print("stopped streaming.")

22/05/11 18:10:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/05/11 18:10:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/11 18:12:30 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 12156 milliseconds


stopped streaming.


In [3]:
spark.stop()
print("stopped spark successfully.")

stopped spark successfully.


# Check dataset

In [6]:
try:
    spark, sc = init_spark()
    test = spark.read.format("csv").option("header", True).load("s3a://reddit-stevenhurwitt/test_technology")
    test_pandas = test.toPandas()

except KeyboardInterrupt:
    print("loading csv took too long... cancelled.")

created spark successfully


/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [8]:
test_pandas

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,pinned96,created_utc,num_crossposts,media,is_video
0,None,technology,None,t2_7ccf,false,None,0,false,'An army of robots' and zero human workers wil...,r/technology,...,/r/technology/comments/unf8aa/an_army_of_robot...,all_ads,false,https://www.asiaone.com/digital/army-robots-an...,11960083,false,1.6522903E9,1,None,false
1,None,technology,None,t2_7ccf,false,None,0,false,'An army of robots' and zero human workers wil...,r/technology,...,/r/technology/comments/unf8aa/an_army_of_robot...,all_ads,false,https://www.asiaone.com/digital/army-robots-an...,11960120,false,1.6522903E9,1,None,false
2,None,technology,None,t2_7ccf,false,None,0,false,'An army of robots' and zero human workers wil...,r/technology,...,/r/technology/comments/unf8aa/an_army_of_robot...,all_ads,false,https://www.asiaone.com/digital/army-robots-an...,11960131,false,1.6522903E9,1,None,false
3,None,technology,None,t2_7ccf,false,None,0,false,'An army of robots' and zero human workers wil...,r/technology,...,/r/technology/comments/unf8aa/an_army_of_robot...,all_ads,false,https://www.asiaone.com/digital/army-robots-an...,11960008,false,1.6522903E9,1,None,false
4,None,technology,None,t2_7ccf,false,None,0,false,'An army of robots' and zero human workers wil...,r/technology,...,/r/technology/comments/unf8aa/an_army_of_robot...,all_ads,false,https://www.asiaone.com/digital/army-robots-an...,11960024,false,1.6522903E9,1,None,false
5,None,technology,None,t2_4g3lx,false,None,0,false,Privacy groups warn about data-tracking if Roe...,r/technology,...,/r/technology/comments/uni84j/privacy_groups_w...,all_ads,false,https://www.nbcnews.com/tech/roe-v-wade-overtu...,11960473,false,1.65229837E9,0,None,false
6,None,technology,None,t2_gsox0fbi,false,None,0,false,Exclusive: Facebook-owner Meta tells hardware ...,r/technology,...,/r/technology/comments/unldgj/exclusive_facebo...,all_ads,false,https://www.reuters.com/technology/exclusive-f...,11961201,false,1.65230694E9,0,None,false
7,None,technology,None,t2_13hs0p,false,None,0,false,Minnesota Wants to Ban Under-18s From User-Gen...,r/technology,...,/r/technology/comments/unhsn8/minnesota_wants_...,all_ads,false,https://blog.ericgoldman.org/archives/2022/05/...,11960381,false,1.65229722E9,0,None,false
8,None,technology,None,t2_gsox0fbi,false,None,0,false,Exclusive: Facebook-owner Meta tells hardware ...,r/technology,...,all_ads,false,https://www.reuters.com/technology/exclusive-f...,11961342,1.65230694E9,0,None,false,None,None
9,None,technology,None,t2_7ccf,false,None,0,false,"Robots Are Writing Poetry, and Many People Can...",r/technology,...,/r/technology/comments/unjm67/robots_are_writi...,all_ads,false,https://thewalrus.ca/ai-poetry/,11960804,false,1.65230221E9,0,None,false


In [15]:
test_pandas.to_csv("sample_response.csv")

22/05/11 19:29:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 6487 milliseconds
22/05/11 19:39:37 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 7039 milliseconds
22/05/11 19:48:00 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 6011 milliseconds
22/05/11 19:58:26 WARN FileStreamSinkLog: Compacting took 4182 ms for compact batch 9
22/05/11 19:58:26 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 8687 milliseconds
22/05/11 20:40:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 6688 milliseconds
22/05/11 20:51:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 7091 milliseconds
22/05/11